In [1]:
import sys
sys.path.append('../scripts')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import helpers as hp
import pickle as pkl
import helpers as hp
import os

from keras.layers import Input, Dense, LSTM, Embedding, Merge
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adadelta
from keras.callbacks import Callback
from keras import backend as K

from sklearn.metrics import f1_score, precision_score, recall_score,\
                            average_precision_score, roc_auc_score,\
                            roc_curve, precision_recall_curve, confusion_matrix,\
                            accuracy_score

from IPython.core.interactiveshell import InteractiveShell
from matplotlib import rcParams
from importlib import reload
from harness import TrainingHarness

rcParams['font.family'] = 'serif'
rcParams['font.serif'] = 'times new roman'

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

Using TensorFlow backend.


# Data Preprocessing

In [2]:
with open('../data/embeddings/glove-300.map', 'rb') as f:
    map = pkl.load(f)

data_dir = os.path.join('..','data')
source_dir = os.path.join(data_dir,'split','amazon-google')
data = hp.load_data(source_dir)

datasets = ['train_1', 'val_1', 'test_1', 'train_2', 'val_2', 'test_2']

# drop columns that do not contain embeddings
data['train_1'] = data['train_1'].drop(['id1', 'price'], axis = 'columns')
data['train_2'] = data['train_2'].drop(['id2', 'price'], axis = 'columns')
data['val_1'] = data['val_1'].drop(['id1', 'price'], axis = 'columns')
data['val_2'] = data['val_2'].drop(['id2', 'price'], axis = 'columns')
data['test_1'] = data['test_1'].drop(['id1', 'price'], axis = 'columns')
data['test_2'] = data['test_2'].drop(['id2', 'price'], axis = 'columns')


# concatenate all embedding columns so each record becomes one giant list
def concat_columns(x):
    x = x['title'] + x['description'] + x['manufacturer']
    return x

for df_name in datasets:
    data[df_name] = data[df_name].apply(concat_columns, axis='columns')
    
# pad all sequences with less than 500 indices with zeros
# limit all sequences to 500 indices
maxlen = 500
for df_name in datasets:
    data[df_name] = pad_sequences(data[df_name], maxlen=maxlen)

# Build Model

In [21]:
glove_matrix = np.load(os.path.join(data_dir, 
                                    'embeddings',
                                    'glove-300.matrix.npy'))

In [23]:
n_hidden = 50

input_left = Input(shape=(maxlen,))
input_right = Input(shape=(maxlen,))

embedding_layer = Embedding(glove_matrix.shape[0],
                            glove_matrix.shape[1],
                            weights=[glove_matrix],
                            input_length=maxlen,
                            trainable=False)
embedded_left = embedding_layer(input_left)
embedded_right = embedding_layer(input_right)

# lstm_layer = LSTM(n_hidden)
# output_left = lstm_layer(embedded_right)
# output_right = lstm_layer(embedded_left)

# def distance_metric(x1, x2):
#     l1 = -K.sum(K.abs(x1 - x2), axis=1, keepdims=True)
#     return K.exp(l1)

# merge_layer = Merge(mode=lambda x: distance_metric(x[0], x[1]),
#                     output_shape=lambda x: (x[0][0], 1))
# distance = merge_layer([output_left, output_right])

malstm = Model([input_left, input_right], [embedded_left, embedded_right])

# Pre-training

In [5]:
compile_args = dict(optimizer='adam', loss='mean_squared_error')
fit_args = dict(x = [data['train_1'], data['train_2']],
                  y = data['train_y'],
                  epochs = 10,
                  batch_size = 2048,
                  validation_data = ([data['val_1'], data['val_2']], data['val_y']))
th = TrainingHarness(malstm,
                     n_checkpoints = 10,
                     compile_args = compile_args,
                     fit_args = fit_args,
                     npy_embedding_matrix = '../data/embeddings/glove-300.matrix.npy')

In [6]:
th.fit(class_weight = {0:1, 1:5})

Train on 10400 samples, validate on 1300 samples
Epoch 1/1
10400/10400 [==============================] - 12s 1ms/step - loss: 0.4919 - val_loss: 0.0820


/home/paperspace/anaconda3/envs/python-36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/paperspace/anaconda3/envs/python-36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 10400 samples, validate on 1300 samples
Epoch 1/1
10400/10400 [==============================] - 12s 1ms/step - loss: 0.4625 - val_loss: 0.0790
Train on 10400 samples, validate on 1300 samples
Epoch 1/1
10400/10400 [==============================] - 12s 1ms/step - loss: 0.4226 - val_loss: 0.0805
Train on 10400 samples, validate on 1300 samples
Epoch 1/1
10400/10400 [==============================] - 12s 1ms/step - loss: 0.3873 - val_loss: 0.0864


KeyboardInterrupt: 

In [11]:
x = LSTM(50)

In [18]:
x.go_backwards

False